In [ ]:
import cv2
import numpy as np
import time
import gc
from skimage.filters import gaussian
from scipy.fftpack import dct
from sklearn.metrics import mean_squared_error

%matplotlib inline
original_path = '../../data/1080p/1HWSFYQXa1Q.mp4'
rendition_path = '../../data/144p/1HWSFYQXa1Q.mp4'

In [ ]:
import subprocess
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

width = 960
height = 540

t0 = time.time()
tp = time.time()
proc_original = subprocess.Popen(['ffmpeg',
    "-i", original_path,
    "-f", "image2pipe",
    "-s", "{}x{}".format(width, height),
    "-pix_fmt", "gray8",
    "-vcodec", "rawvideo",
    "-"
    ],
    stdout=subprocess.PIPE,
)

proc_rendition = subprocess.Popen(['ffmpeg',
    "-i", rendition_path,
    "-f", "image2pipe",
    #"-vf", "select=mod(n-1\,2)",
    "-s", "{}x{}".format(width, height),
    "-pix_fmt", "gray8",
    "-vcodec", "rawvideo",
    "-"],
    stdout=subprocess.PIPE,
)

np_list_original = []
np_list_rendition = []
fps = 30
n_secs = 2
max_frames = fps * n_secs
max_samples = max_frames
random_sampler = list(np.random.choice(max_frames, max_samples, replace=False))

print(random_sampler)
i=0
while True:
    raw_image_original = proc_original.stdout.read(width * height)
    raw_image_rendition = proc_rendition.stdout.read(width * height)
    if not raw_image_original or len(np_list_original) > max_frames:
        break
        
    if i in random_sampler:
        image_original = np.frombuffer(raw_image_original, dtype='uint8')
        image_rendition = np.frombuffer(raw_image_rendition, dtype='uint8')
    
        np_list_original.append(image_original)
        np_list_rendition.append(image_rendition)
    i += 1
proc_original.stdout.flush()
proc_rendition.stdout.flush()

del image_original, image_rendition, raw_image_original, raw_image_rendition


np_array_original = np.asarray(np_list_original).reshape((len(np_list_original),height, width))
np_array_rendition = np.asarray(np_list_rendition).reshape((len(np_list_rendition),height, width))



del np_list_original, np_list_rendition

print('Shape = ', np_array_original.shape)
print('Elapsed decoding=', time.time()-tp)

tp = time.time()
np.save('original.npy', np_array_original)
print('Elapsed saving=', time.time()-tp)


tp = time.time()
n_jobs = 8
gaussian_array_original = Parallel(n_jobs=n_jobs,  prefer="threads")(
    delayed(gaussian)(x, sigma=4)
    for x in np.array_split(np_array_original,n_jobs))

gaussian_array_rendition = Parallel(n_jobs=n_jobs,  prefer="threads")(
    delayed(gaussian)(x, sigma=4)
    for x in np.array_split(np_array_rendition,n_jobs))

original = []
rendition = []
print('Elapsed joblib={}, jobs={}'.format(time.time()-tp, n_jobs))

tp = time.time()

mse = []
difference = []

for i in range(n_jobs):
    for j in range(gaussian_array_original[i].shape[0]):

        gaussian_difference = np.float32(gaussian_array_original[i][j] - 
                                   gaussian_array_rendition[i][j])
        
        difference.append(np.sum(np.abs(gaussian_difference)))
        mse.append(np.mean(gaussian_difference**2))
print(difference)
print(np.mean(difference), np.mean(mse))
print('Elapsed gaussian_diference={}, jobs={}'.format(time.time()-tp, n_jobs))

print('Elapsed=', time.time()-t0)

del np_array_original, np_array_rendition, gaussian_array_original, gaussian_array_rendition
del Parallel

gc.collect()